In [19]:
import xarray as xr
import xclim as xc
from tqdm import tqdm

# เปิดไฟล์ NetCDF
ds_tmin = xr.open_dataset("../dataset-nc/TH_tmin_ERA5_day.1960-2022.nc")
ds_tmax = xr.open_dataset("../dataset-nc/TH_tmax_ERA5_day.1960-2022.nc")
ds_pr = xr.open_dataset("../dataset-nc/TH_pr_ERA5_day.1960-2022.nc")

# แปลงค่าจากเคลวินเป็นองศาเซลเซียส และ mm/day
ds_tmin['mn2t'] = ds_tmin['mn2t'] - 273.15
ds_tmax['mx2t'] = ds_tmax['mx2t'] - 273.15
ds_pr['tp'] = ds_pr['tp'] * 1000  # m -> mm/day

# กำหนด units
ds_tmin['mn2t'].attrs['units'] = 'degC'  # องศาเซลเซียส
ds_tmax['mx2t'].attrs['units'] = 'degC'  # องศาเซลเซียส
ds_pr['tp'].attrs['units'] = 'mm/day'  # มิลลิเมตรต่อวัน

# เลือกข้อมูลปี 1960
ds_tmin = ds_tmin.sel(time=slice('1960', '1960'))
ds_tmax = ds_tmax.sel(time=slice('1960', '1960'))
ds_pr = ds_pr.sel(time=slice('1960', '1960'))

# เตรียมรายการสำหรับเก็บผลลัพธ์
results_all_grids = []

# ใช้ tqdm เพื่อแสดง progress bar
total_grids = len(ds_tmin.latitude) * len(ds_tmin.longitude)
print("Processing data for all grids in the year 1960...")
with tqdm(total=total_grids, desc="Processing Grids") as pbar:
    for lat in ds_tmin.latitude:
        for lon in ds_tmin.longitude:
            # ดึงข้อมูลสำหรับแต่ละ grid
            grid_data_tmin = ds_tmin['mn2t'].sel(latitude=lat, longitude=lon, method='nearest')
            grid_data_tmax = ds_tmax['mx2t'].sel(latitude=lat, longitude=lon, method='nearest')
            grid_data_pr = ds_pr['tp'].sel(latitude=lat, longitude=lon, method='nearest')

            # กำหนด units ให้กับข้อมูลแต่ละ grid
            grid_data_tmin.attrs['units'] = 'degC'
            grid_data_tmax.attrs['units'] = 'degC'
            grid_data_pr.attrs['units'] = 'mm/day'

            # คำนวณค่า TNn, TXx, RX1day แบบ yearly
            tnn_yearly = xc.indicators.atmos.tn_min(grid_data_tmin, freq='YS')
            txx_yearly = xc.indicators.atmos.tx_max(grid_data_tmax, freq='YS')
            rx1day_yearly = xc.indices.max_1day_precipitation_amount(grid_data_pr, freq='YS')

            # หาวันที่ TNn, TXx, RX1day
            tnn_date = grid_data_tmin.time[grid_data_tmin.argmin().item()].values
            txx_date = grid_data_tmax.time[grid_data_tmax.argmax().item()].values
            rx1day_date = grid_data_pr.time[grid_data_pr.argmax().item()].values

            # แปลง TNn และ TXx จากเคลวินเป็นเซลเซียส
            tnn_celsius = tnn_yearly.values.item() - 273.15 if tnn_yearly else None
            txx_celsius = txx_yearly.values.item() - 273.15 if txx_yearly else None

            # คำนวณรายเดือน
            for month in range(1, 13):
                # เลือกข้อมูลรายเดือน
                data_tmin_monthly = grid_data_tmin.sel(time=f'1960-{month:02d}')
                data_tmax_monthly = grid_data_tmax.sel(time=f'1960-{month:02d}')
                data_pr_monthly = grid_data_pr.sel(time=f'1960-{month:02d}')

                # คำนวณค่าเฉลี่ยหรือผลรวม
                tmin_monthly = data_tmin_monthly.mean().values if len(data_tmin_monthly) > 0 else None
                tmax_monthly = data_tmax_monthly.mean().values if len(data_tmax_monthly) > 0 else None
                pr_monthly = data_pr_monthly.sum().values if len(data_pr_monthly) > 0 else None

                # เก็บผลลัพธ์
                results_all_grids.append({
                    'latitude': lat.item(),
                    'longitude': lon.item(),
                    'year': 1960,
                    'month': month,
                    'tmin': tmin_monthly,
                    'tmax': tmax_monthly,
                    'pr': pr_monthly,
                    'txx': txx_celsius,
                    'tnn': tnn_celsius,
                    'rx1day': rx1day_yearly.values.item() if rx1day_yearly else None,
                    'tnn_date': tnn_date,
                    'txx_date': txx_date,
                    'rx1day_date': rx1day_date
                })

            # อัปเดต progress bar
            pbar.update(1)

# แสดงผลลัพธ์ตัวอย่าง
print("\nSample Results:")
for result in results_all_grids[:5]:  # แสดงเฉพาะ 5 รายการแรก
    print(result)

# import xarray as xr
# import xclim as xc
# import pandas as pd

# # เปิดไฟล์ NetCDF
# ds_tmin = xr.open_dataset("../dataset-nc/TH_tmin_ERA5_day.1960-2022.nc")
# ds_tmax = xr.open_dataset("../dataset-nc/TH_tmax_ERA5_day.1960-2022.nc")
# ds_pr = xr.open_dataset("../dataset-nc/TH_pr_ERA5_day.1960-2022.nc")

# # เลือกตำแหน่ง grid ที่ต้องการ
# latitude = ds_tmin.latitude[0].item()
# longitude = ds_tmin.longitude[0].item()

# # ดึงข้อมูลสำหรับ grid ที่เลือก
# grid_data_tmin = ds_tmin['mn2t'].sel(latitude=latitude, longitude=longitude, method='nearest')
# grid_data_tmax = ds_tmax['mx2t'].sel(latitude=latitude, longitude=longitude, method='nearest')
# grid_data_pr = ds_pr['tp'].sel(latitude=latitude, longitude=longitude, method='nearest')

# # แปลงค่าจากเคลวินเป็นองศาเซลเซียส
# grid_data_tmin_celsius = grid_data_tmin - 273.15
# grid_data_tmax_celsius = grid_data_tmax - 273.15
# grid_data_pr_mm = grid_data_pr * 1000  # เปลี่ยนจาก m/day เป็น mm/day

# # กำหนด units เป็น 'degC' และ 'mm/day'
# grid_data_tmin_celsius.attrs['units'] = 'degC'
# grid_data_tmax_celsius.attrs['units'] = 'degC'
# grid_data_pr_mm.attrs['units'] = 'mm/day'

# # เลือกข้อมูลสำหรับปี 1960
# data_tmin = grid_data_tmin_celsius.sel(time=slice('1960', '1960'))
# data_tmax = grid_data_tmax_celsius.sel(time=slice('1960', '1960'))
# data_pr = grid_data_pr_mm.sel(time=slice('1960', '1960')) 

# # คำนวณ TNn และ TXx แบบ yearly
# tnn_yearly = xc.indicators.atmos.tn_min(data_tmin, freq='YS') - 273.15  # Yearly minimum
# txx_yearly = xc.indicators.atmos.tx_max(data_tmax, freq='YS') - 273.15  # Yearly maximum

# # คำนวณ RX1day แบบ yearly (ใช้ max_1day_precipitation_amount แทน rx1day)
# rx1day_yearly = xc.indices.max_1day_precipitation_amount(data_pr, freq='Y').isel(time=0).values 

# # หาวันที่ที่มีค่า TNn ต่ำสุดในปี 1960
# tnn_dates_yearly = data_tmin.groupby('time.year').apply(lambda x: x.time[x.argmin().item()])

# # หาวันที่มีค่า TXx สูงสุดในปี 1960
# txx_dates_yearly = data_tmax.groupby('time.year').apply(lambda x: x.time[x.argmax().item()])

# # หาวันที่มีค่า RX1day สูงสุดในปี 1960
# rx1day_dates_yearly = data_pr.groupby('time.year').apply(lambda x: x.time[x.argmax().item()])

# # สร้างรายการเก็บผลลัพธ์
# results_monthly = []
# for month in range(1, 13):
#     # เลือกข้อมูลรายเดือน
#     data_tmin_monthly = data_tmin.sel(time=f'1960-{month:02d}')
#     data_tmax_monthly = data_tmax.sel(time=f'1960-{month:02d}')
#     data_pr_monthly = data_pr.sel(time=f'1960-{month:02d}')

#     # คำนวณค่า tmin, tmax สำหรับเดือนนั้น
#     tmin_monthly = data_tmin_monthly.mean().values
#     tmax_monthly = data_tmax_monthly.mean().values
#     pr_monthly = data_pr_monthly.sum().values 
    
#     # ค่า txx, tnn, rx1day สำหรับปี 1960
#     tnn_year = tnn_yearly.sel(time='1960').values.item()
#     txx_year = txx_yearly.sel(time='1960').values.item()
#     rx1day_year = rx1day_yearly.item()
    
#     # หาวันที่ของ TNn, TXx และ RX1day
#     tnn_date = tnn_dates_yearly.sel(year=1960).values
#     txx_date = txx_dates_yearly.sel(year=1960).values
#     rx1day_date = rx1day_dates_yearly.sel(year=1960).values 
    
#     # เก็บผลลัพธ์
#     results_monthly.append({
#         'year': 1960,
#         'month': month,
#         'tmin': tmin_monthly,
#         'tmax': tmax_monthly,
#         'pr': pr_monthly,
#         'txx': txx_year,
#         'tnn': tnn_year,
#         'rx1day': rx1day_year,
#         'tnn_date': tnn_date,
#         'txx_date': txx_date,
#         'rx1day_date': rx1day_date
#     })

# # แสดงผลลัพธ์
# print("\nMonthly TMin, TMax, Precipitation, and Yearly TNn, TXx, RX1day for the year 1960:")
# for result in results_monthly:
#     print(f"Grid Position: A, Year: {result['year']} Month: {result['month']:02d}, "
#           f"TMin: {result['tmin']:.2f}°C, TMax: {result['tmax']:.2f}°C, "
#           f"Precipitation: {result['pr']:.2f} mm/day, "
#           f"TXx (Yearly): {result['txx']:.2f}°C, Tnn (Yearly): {result['tnn']:.2f}°C, "
#           f"RX1day (Yearly): {result['rx1day']:.2f} mm/day, "
#           f"TNn Date: {result['tnn_date']}, TXx Date: {result['txx_date']}, RX1day Date: {result['rx1day_date']}")


Processing data for all grids in the year 1960...


Processing Grids:   0%|          | 0/3577 [00:00<?, ?it/s]C:\Users\supakron\AppData\Roaming\Python\Python312\site-packages\xclim\core\cfchecks.py:79: UserWarning: Variable does not have a `cell_methods` attribute.
  _check_cell_methods(
C:\Users\supakron\AppData\Roaming\Python\Python312\site-packages\xclim\core\cfchecks.py:83: UserWarning: Variable does not have a `standard_name` attribute.
  check_valid(vardata, "standard_name", data["standard_name"])
C:\Users\supakron\AppData\Roaming\Python\Python312\site-packages\xclim\core\cfchecks.py:79: UserWarning: Variable does not have a `cell_methods` attribute.
  _check_cell_methods(
C:\Users\supakron\AppData\Roaming\Python\Python312\site-packages\xclim\core\cfchecks.py:83: UserWarning: Variable does not have a `standard_name` attribute.
  check_valid(vardata, "standard_name", data["standard_name"])
Processing Grids:   0%|          | 1/3577 [00:00<13:19,  4.47it/s]C:\Users\supakron\AppData\Roaming\Python\Python312\site-packages\xclim\core\cf


Sample Results:
{'latitude': 22.0, 'longitude': 95.0, 'year': 1960, 'month': 1, 'tmin': array(20.59296102), 'tmax': array(21.39638984), 'pr': array(0.01291064), 'txx': 35.69512895772294, 'tnn': 18.925585864667198, 'rx1day': 29.33652426473859, 'tnn_date': np.datetime64('1960-01-01T11:00:00.000000000'), 'txx_date': np.datetime64('1960-04-30T11:00:00.000000000'), 'rx1day_date': np.datetime64('1960-09-22T11:00:00.000000000')}
{'latitude': 22.0, 'longitude': 95.0, 'year': 1960, 'month': 2, 'tmin': array(23.68533295), 'tmax': array(24.66872404), 'pr': array(1.20481974), 'txx': 35.69512895772294, 'tnn': 18.925585864667198, 'rx1day': 29.33652426473859, 'tnn_date': np.datetime64('1960-01-01T11:00:00.000000000'), 'txx_date': np.datetime64('1960-04-30T11:00:00.000000000'), 'rx1day_date': np.datetime64('1960-09-22T11:00:00.000000000')}
{'latitude': 22.0, 'longitude': 95.0, 'year': 1960, 'month': 3, 'tmin': array(27.19652798), 'tmax': array(28.22830723), 'pr': array(1.53937999), 'txx': 35.69512895

In [ ]:
import json
import xarray as xr
import xclim as xc
from tqdm import tqdm

# เปิดไฟล์ NetCDF
ds_tmin = xr.open_dataset("../dataset-nc/TH_tmin_ERA5_day.1960-2022.nc")
ds_tmax = xr.open_dataset("../dataset-nc/TH_tmax_ERA5_day.1960-2022.nc")
ds_pr = xr.open_dataset("../dataset-nc/TH_pr_ERA5_day.1960-2022.nc")

# แปลงค่าจากเคลวินเป็นองศาเซลเซียส และ mm/day
ds_tmin['mn2t'] = ds_tmin['mn2t'] - 273.15
ds_tmax['mx2t'] = ds_tmax['mx2t'] - 273.15
ds_pr['tp'] = ds_pr['tp'] * 1000  # m -> mm/day

# กำหนด units
ds_tmin['mn2t'].attrs['units'] = 'degC'  # องศาเซลเซียส
ds_tmax['mx2t'].attrs['units'] = 'degC'  # องศาเซลเซียส
ds_pr['tp'].attrs['units'] = 'mm/day'  # มิลลิเมตรต่อวัน

# เลือกข้อมูลปี 1960
ds_tmin = ds_tmin.sel(time=slice('1960', '1960'))
ds_tmax = ds_tmax.sel(time=slice('1960', '1960'))
ds_pr = ds_pr.sel(time=slice('1960', '1960'))

# เตรียมรายการสำหรับ GeoJSON
geojson_features = []

# ใช้ tqdm เพื่อแสดง progress bar
total_grids = len(ds_tmin.latitude) * len(ds_tmin.longitude)
print("Processing data for all grids in the year 1960...")
with tqdm(total=total_grids, desc="Processing Grids") as pbar:
    for lat in ds_tmin.latitude:
        for lon in ds_tmin.longitude:
            # ดึงข้อมูลสำหรับแต่ละ grid
            grid_data_tmin = ds_tmin['mn2t'].sel(latitude=lat, longitude=lon, method='nearest')
            grid_data_tmax = ds_tmax['mx2t'].sel(latitude=lat, longitude=lon, method='nearest')
            grid_data_pr = ds_pr['tp'].sel(latitude=lat, longitude=lon, method='nearest')

            # คำนวณค่า TNn, TXx, RX1day แบบ yearly
            tnn_yearly = xc.indicators.atmos.tn_min(grid_data_tmin, freq='YS')
            txx_yearly = xc.indicators.atmos.tx_max(grid_data_tmax, freq='YS')
            rx1day_yearly = xc.indices.max_1day_precipitation_amount(grid_data_pr, freq='YS')

            # คำนวณรายเดือน
            for month in range(1, 13):
                # เลือกข้อมูลรายเดือน
                data_tmin_monthly = grid_data_tmin.sel(time=f'1960-{month:02d}')
                data_tmax_monthly = grid_data_tmax.sel(time=f'1960-{month:02d}')
                data_pr_monthly = grid_data_pr.sel(time=f'1960-{month:02d}')

                # คำนวณค่าเฉลี่ยหรือผลรวม
                tmin_monthly = data_tmin_monthly.mean().values if len(data_tmin_monthly) > 0 else None
                tmax_monthly = data_tmax_monthly.mean().values if len(data_tmax_monthly) > 0 else None
                pr_monthly = data_pr_monthly.sum().values if len(data_pr_monthly) > 0 else None

                # สร้าง geometry สำหรับ grid
                geometry = {
                    "type": "Polygon",
                    "coordinates": [[
                        [lon - 0.125, lat - 0.125],
                        [lon + 0.125, lat - 0.125],
                        [lon + 0.125, lat + 0.125],
                        [lon - 0.125, lat + 0.125],
                        [lon - 0.125, lat - 0.125]
                    ]]
                }

                # สร้าง properties
                properties = {
                    "tmax": float(tmax_monthly) if tmax_monthly is not None else None,
                    "tmin": float(tmin_monthly) if tmin_monthly is not None else None,
                    "pre": float(pr_monthly) if pr_monthly is not None else None,
                    "txx": float(txx_yearly.values) if txx_yearly else None,
                    "tnn": float(tnn_yearly.values) if tnn_yearly else None,
                    "rx1day": float(rx1day_yearly.values) if rx1day_yearly else None,
                    "month": month
                }

                # เพิ่มเข้าไปใน geojson features
                geojson_features.append({
                    "type": "Feature",
                    "geometry": geometry,
                    "properties": properties
                })

            # อัปเดต progress bar
            pbar.update(1)

# สร้าง GeoJSON FeatureCollection
geojson_data = {
    "type": "FeatureCollection",
    "features": geojson_features
}

# บันทึกลงไฟล์ GeoJSON
output_file = "era_data_grid_1960.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(geojson_data, f, ensure_ascii=False, indent=4)

print(f"GeoJSON data has been saved to {output_file}")
